In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMG_SIZE = (224, 224)

train_dir = "Class25_Dataset/train/"
test_dir = "Class25_Dataset/test/"



train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)



Found 338 files belonging to 25 classes.
Found 71 files belonging to 25 classes.


In [3]:
print(train_data)

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 25), dtype=tf.float32, name=None))>


In [4]:
checkpoint_path = "better_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True, # save only the model weights
                                                         monitor="val_accuracy", # save the model weights which score the best validation accuracy
                                                         save_best_only=True)

In [44]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# original_data = Sequential([
#   preprocessing.RandomRotation(0), 
# ], name="original_data")

data_augmentation = Sequential([
  preprocessing.RandomRotation(0),
  preprocessing.RandomFlip("horizontal"),
  preprocessing.RandomRotation(0.2), 
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomContrast(0.1)
], name="data_augmentation")

In [45]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False


inputs = layers.Input(shape=(224, 224, 3), name="input_layer")

x = data_augmentation(inputs)
x = base_model(x, training=False) 
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) 
outputs = layers.Dense(len(train_data.class_names), activation="softmax", name="output_layer")(x) 
model = tf.keras.Model(inputs, outputs)

In [46]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 data_augmentation (Sequenti  (None, 224, 224, 3)      0         
 al)                                                             
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling (Glo  (None, 1280)             0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 25)                32025     
                                                                 
Total params: 4,081,596
Trainable params: 32,025
Non-traina

In [47]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit
history_base_model = model.fit(train_data,
                                           epochs=21, 
                                           validation_data=test_data,
                                           validation_steps=len(test_data))

Epoch 1/21
11/11 [==============================] - 19s 1s/step - loss: 3.2372 - accuracy: 0.0888 - val_loss: 2.9894 - val_accuracy: 0.1408
Epoch 2/21
11/11 [==============================] - 13s 1s/step - loss: 2.7392 - accuracy: 0.2722 - val_loss: 2.6848 - val_accuracy: 0.2113
Epoch 3/21
11/11 [==============================] - 13s 1s/step - loss: 2.3650 - accuracy: 0.4408 - val_loss: 2.4439 - val_accuracy: 0.3803
Epoch 4/21
11/11 [==============================] - 12s 1s/step - loss: 2.0877 - accuracy: 0.5917 - val_loss: 2.2546 - val_accuracy: 0.3803
Epoch 5/21
11/11 [==============================] - 13s 1s/step - loss: 1.8498 - accuracy: 0.6420 - val_loss: 2.1243 - val_accuracy: 0.3944
Epoch 6/21
11/11 [==============================] - 12s 1s/step - loss: 1.6290 - accuracy: 0.7367 - val_loss: 2.0058 - val_accuracy: 0.4507
Epoch 7/21
11/11 [==============================] - 12s 1s/step - loss: 1.5059 - accuracy: 0.7574 - val_loss: 1.8942 - val_accuracy: 0.4648
Epoch 8/21
11/11 [==

In [48]:
base_model.trainable = True

# Refreeze every layer except for the last 5
for layer in base_model.layers[:-5]:
  layer.trainable = False


In [49]:
new_model = model

In [50]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # 10x lower learning rate than default
              metrics=['accuracy'])

In [51]:
fine_tune_epochs = 31

history_model_fine_tune = model.fit(train_data,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=len(test_data),
                                                     initial_epoch=history_base_model.epoch[-1])

Epoch 21/31
11/11 [==============================] - 19s 1s/step - loss: 0.5247 - accuracy: 0.9379 - val_loss: 1.3619 - val_accuracy: 0.6056
Epoch 22/31
11/11 [==============================] - 15s 1s/step - loss: 0.4282 - accuracy: 0.9527 - val_loss: 1.3270 - val_accuracy: 0.6338
Epoch 23/31
11/11 [==============================] - 14s 1s/step - loss: 0.3602 - accuracy: 0.9586 - val_loss: 1.2966 - val_accuracy: 0.6338
Epoch 24/31
11/11 [==============================] - 13s 1s/step - loss: 0.3294 - accuracy: 0.9675 - val_loss: 1.2420 - val_accuracy: 0.6338
Epoch 25/31
11/11 [==============================] - 13s 1s/step - loss: 0.3222 - accuracy: 0.9527 - val_loss: 1.2048 - val_accuracy: 0.6620
Epoch 26/31
11/11 [==============================] - 41s 4s/step - loss: 0.2330 - accuracy: 0.9822 - val_loss: 1.1914 - val_accuracy: 0.6901
Epoch 27/31
11/11 [==============================] - 49s 4s/step - loss: 0.2293 - accuracy: 0.9793 - val_loss: 1.1789 - val_accuracy: 0.6761
Epoch 28/31
1

In [52]:
import cv2
import numpy as np
IMAGE_SHAPE = (224, 224)
IMAGE_SHAPE+(3,)
test_dir = "C:\\Users\\bhave\\Project-1\\Class25_Dataset\\train\\a\\2.jpeg"
img = cv2.imread(test_dir)
img_resize = cv2.resize(img , IMAGE_SHAPE)
predicted = model.predict(np.array([img_resize]))
ind = np.argmax(predicted , axis = 1)
index = ind[0]
print(index)
cv2.imshow('detected', img)
cv2.waitKey(0)

18


-1

In [53]:
model_json = model.to_json()
with open("better_model.json" , "w") as json_file:
    json_file.write(model_json)
model.save_weights("better_model_weights.h5")

In [54]:
from keras.models import model_from_json
import json

model_file = open("better_model.json", "r")
model_json = model_file.read()
model = model_from_json(model_json)
model.load_weights("better_model_weights.h5")